# PMIX SBX

In [9]:
from configparser import ConfigParser

from snowflake.connector import connect #type: ignore

from sklearn.model_selection import train_test_split #type: ignore
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score 

from pandas import DataFrame, Series, merge, concat

import json

from matplotlib import pyplot as plt

import numpy as np

from datetime import datetime

config = ConfigParser()
config.read('config.ini')

conn = connect(
    user = config.get('SNOWFLAKE', 'USER'),
    password = config.get('SNOWFLAKE', 'PASSWORD'),
    account = config.get('SNOWFLAKE', 'ACCOUNT'),
    database = config.get('SNOWFLAKE', 'DATABASE'),
    warehouse = config.get('SNOWFLAKE', 'WAREHOUSE'),
    schema = config.get('SNOWFLAKE', 'SCHEMA'),
    role = config.get('SNOWFLAKE', 'ROLE'),
)

FECHA_INICIO_ACTIVOS = "'2023-06-12'"
FECHA_INICIO = "'2023-09-04'"
FECHA_FIN = "'2023-09-10'"
ANIO_MEDICION = 2023
SEMANA_MEDICION = 36

## Resultados Journey Nuevos:

In [10]:
resultados_journey_nuevos = conn.cursor().execute("""
    WITH 

    ENVIADOS AS (
        SELECT
            FOLIO_CUPON,
            USER_ID,
            to_date(max(TIMESTAMP)) AS FECHA,
            STEP
        FROM
            SEGMENT_EVENTS.HTTP_REGISTRA_CUPON.COUPON_RECORD AS ENVIADOS
        WHERE
            SOURCE = 'SBX Journey Nuevos'
        GROUP BY
            FOLIO_CUPON,
            USER_ID,
            STEP
        HAVING
            FECHA >= to_date('2023-07-10')
    )

    SELECT
        ANIO_ALSEA,
        SEM_ALSEA,
        count(REDIMIDOS.MONTO_VENTA) AS REDENCIONES,
        sum(REDIMIDOS.MONTO_VENTA) / 1.16 AS VENTA_CON_REDENCION
    FROM
        ENVIADOS 
    LEFT JOIN
        SEGMENT_EVENTS.CUPONERA_ALSEA_PROD.REDEMPTION_COUPON AS REDIMIDOS
    ON
        ENVIADOS.FOLIO_CUPON = REDIMIDOS.ID_COUPON
    INNER JOIN
        WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME
    ON
        to_date(REDIMIDOS.TIMESTAMP) = DS_DIM_TIME.FECHA
    GROUP BY
        ANIO_ALSEA,
        SEM_ALSEA
    ORDER BY
        ANIO_ALSEA DESC,
        SEM_ALSEA DESC
    ;
""")

resultados_journey_nuevos = resultados_journey_nuevos.fetch_pandas_all()

print(resultados_journey_nuevos.head())

   ANIO_ALSEA  SEM_ALSEA  REDENCIONES  VENTA_CON_REDENCION
0        2023         37           25          2372.413793
1        2023         36          404         39293.534483
2        2023         35          715         68775.431034
3        2023         34          718         71605.603448
4        2023         33          838         82298.275862


## Resultados WIFI

In [11]:
resultados_wifi = conn.cursor().execute("""
    WITH

    NOSR AS (
        SELECT DISTINCT 
            PROGRAM_WELCOMING.EMAIL,
            to_date(dateadd(hour, -5, PROGRAM_WELCOMING.UUID_TS)) AS CREATED
        FROM 
            SEGMENT_EVENTS.PERSONAS_STARBUCKS_PERSONAS.IDENTIFIES
        INNER JOIN 
            SEGMENT_EVENTS.COREALSEASBX_PROD.PROGRAM_WELCOMING 
        ON
            lower(PROGRAM_WELCOMING.EMAIL) = lower(IDENTIFIES.USER_ID)
        WHERE 
            IDENTIFIES.CONTEXT_PERSONAS_COMPUTATION_KEY LIKE ('j_o_sbx_02_neowifi_v_%')
    ),

    PRIMERA_VENTA AS (
        SELECT DISTINCT
            EMAIL,
            first_value(CHECK_AMOUNT / 1.16) OVER (PARTITION BY EMAIL ORDER BY TRANSACTION_DATE) AS VENTAS,
            first_value(TRANSACTION_DATE)    OVER (PARTITION BY EMAIL ORDER BY TRANSACTION_DATE) AS FECHA
        FROM 
            "SEGMENT_EVENTS"."SESSIONM_SBX"."SM_TRANSACTION_HEADERS" TXN
        INNER JOIN 
        (
            SELECT DISTINCT 
                TRANSACTION_ID, 
                USER_ID
            FROM 
                "SEGMENT_EVENTS"."SESSIONM_SBX"."SM_TRANSACTION_PAYMENTS"
        ) AS Q
        USING(
            TRANSACTION_ID
        )
        INNER JOIN
            SEGMENT_EVENTS.SESSIONM_SBX.SM_USERS
        USING(
            USER_ID
        )
        WHERE 
            TXN.CHANNEL<> 'API'
        AND
            TXN.TRANSACTION_ID NOT IN 
            (
                SELECT DISTINCT  
                    TRANSACTION_ID 
                FROM 
                    SEGMENT_EVENTS.SESSIONM_SBX.SM_TRANSACTION_ITEMS
                WHERE 
                    POS_ITEM_KEY IN (
                        'MEXLEG_106_sbx',
                        'MEXLEG_109_sbx'
                    ) 
            )
    )

    SELECT 
        ANIO_ALSEA,
        SEM_ALSEA,
        count(DISTINCT PRIMERA_VENTA.EMAIL) AS USR_PRIMERA_VENTA,
        sum(PRIMERA_VENTA.VENTAS) AS VENTAS
    FROM 
        NOSR
    INNER JOIN
        PRIMERA_VENTA
    ON
        lower(NOSR.EMAIL) = lower(PRIMERA_VENTA.EMAIL)
    INNER JOIN
        WOW_REWARDS.WORK_SPACE_WOW_REWARDS.DS_DIM_TIME AS TIME
    ON 
        to_date(PRIMERA_VENTA.FECHA) = TIME.FECHA
    GROUP BY
        ANIO_ALSEA,
        SEM_ALSEA
    ORDER BY 
        ANIO_ALSEA DESC,
        SEM_ALSEA DESC
    ;
""")

resultados_wifi = resultados_wifi.fetch_pandas_all()

print(resultados_wifi.head())

   ANIO_ALSEA  SEM_ALSEA  USR_PRIMERA_VENTA               VENTAS
0        2023         37                 57    9641.379310344827
1        2023         36               1025  137690.250000000004
2        2023         35                990  142053.750000000003
3        2023         34                939  134704.310344827593
4        2023         33               1201  169754.991379310330


## Venta incremental

### Obtenemos la segmentacion que actualmente existe en sessionM, junto con sus datos

In [12]:
cur = conn.cursor().execute(f"""
    WITH

    SEGMENTOS AS (
        SELECT
            SEGMENTO,
            EMAIL
        FROM
            SEGMENT_EVENTS.SESSIONM_SBX.SBX_SEGMENTOS_20230803
        INNER JOIN
            SEGMENT_EVENTS.SESSIONM_SBX.SM_USERS
        USING(
            USER_ID
        )
    )

    SELECT 
        SUM(CHECK_AMOUNT/1.16) AS VTA, 
        COUNT(DISTINCT TRANSACTION_ID) AS ORD,
        to_date({FECHA_FIN}) - to_date(max(CREATED_AT)) AS RECENCY,
        SEGMENTO
    FROM 
        SEGMENT_EVENTS.SESSIONM_SBX.FACT_TRANSACTIONS 
    INNER JOIN
        SEGMENTOS
    ON
        SEGMENTOS.EMAIL = FACT_TRANSACTIONS.EMAIL
    WHERE 
        TRANSACTION_DATE BETWEEN to_date({FECHA_INICIO}) AND to_date({FECHA_FIN})
    GROUP BY
        FACT_TRANSACTIONS.EMAIL,
        SEGMENTO
    ;
""")

df = cur.fetch_pandas_all()

### Obtenemos los datos de los usuarios que no tienen segmento asignado en sessionM

In [13]:
cur_unlabeled = conn.cursor().execute(f"""
    WITH

    SEGMENTOS AS (
        SELECT
            SEGMENTO,
            EMAIL
        FROM
            SEGMENT_EVENTS.SESSIONM_SBX.SBX_SEGMENTOS_20230803
        INNER JOIN
            SEGMENT_EVENTS.SESSIONM_SBX.SM_USERS
        USING(
            USER_ID
        )
    )

    SELECT 
        FACT_TRANSACTIONS.EMAIL,
        SUM(CHECK_AMOUNT/1.16) AS VTA, 
        COUNT(DISTINCT TRANSACTION_ID) AS ORD,
        to_date({FECHA_FIN}) - to_date(max(CREATED_AT)) AS RECENCY
    FROM 
        SEGMENT_EVENTS.SESSIONM_SBX.FACT_TRANSACTIONS 
    LEFT JOIN
        SEGMENTOS
    ON
        SEGMENTOS.EMAIL = FACT_TRANSACTIONS.EMAIL
    WHERE 
        TRANSACTION_DATE BETWEEN to_date({FECHA_INICIO}) AND to_date({FECHA_FIN})
    AND
        SEGMENTOS.EMAIL IS null
    GROUP BY
        FACT_TRANSACTIONS.EMAIL,
        SEGMENTO
    ;
""")

df_unlabeled = cur_unlabeled.fetch_pandas_all() #type: ignore

### Resultados antes de dividir por segmento:

In [14]:
clv_promocion = df['VTA'].mean()

clv_control = df_unlabeled['VTA'].mean()

lift = (clv_promocion / clv_control) - 1

venta_grupo_promocion = df['VTA'].sum()

venta_incremental = lift * int(venta_grupo_promocion)

print(f'CLV grupo promocion:{clv_promocion}')

print(f'CLV grupo control:{clv_control}')

print(f'Lift:{lift}')

print(f'Venta incremental:{venta_incremental}')

CLV grupo promocion:239.13066496975895
CLV grupo control:226.27697217123307
Lift:0.05680512990424402
Venta incremental:3349097.8191759014


### Entrenamos el clasificador y obtenemos los resultados por segmento:

In [15]:
x_unlabeled = df_unlabeled.drop('EMAIL', axis = 1)

x:DataFrame = df.drop('SEGMENTO', axis = 1)
y = df['SEGMENTO']

scaler = StandardScaler()
x = scaler.fit_transform(x)
x_unlabeled = scaler.transform(x_unlabeled)

knn = KNeighborsClassifier(n_neighbors = 1)

knn.fit(x, y)

prediction = DataFrame(knn.predict(x_unlabeled))

df_unlabeled['SEGMENTO'] = prediction[0]

clv_promocion_por_segmento = df.groupby('SEGMENTO').agg({'VTA': 'mean'})

venta_promocion_por_segmento = df.groupby('SEGMENTO').agg({'VTA': 'sum'})

clv_control_por_segmento = df_unlabeled.groupby('SEGMENTO').agg({'VTA': 'mean'})

merged = merge(clv_promocion_por_segmento, clv_control_por_segmento, on = 'SEGMENTO')

merged = merged.rename(columns = {'VTA_x' : 'CLV_PROMOCION', 'VTA_y' : 'CLV_CONTROL'})

merged['CLV_CONTROL'] = merged['CLV_CONTROL'].apply(lambda x : float(x))

merged['LIFT'] = (merged['CLV_PROMOCION'] - merged['CLV_CONTROL']) / merged['CLV_CONTROL']

merged = merge(merged, venta_promocion_por_segmento, on = 'SEGMENTO')

merged['VTA'] = merged['VTA'].apply(lambda x : float(x))

merged['VENTA_INCREMENTAL'] = merged['VTA'] * merged['LIFT']

venta_incremental_asignada = merged['VENTA_INCREMENTAL'].sum()
venta_incremental_por_asignar = venta_incremental - venta_incremental_asignada

merged['PORCENTAJE_VENTA_INCREMENTAL'] = merged['VENTA_INCREMENTAL'] / venta_incremental_asignada 

merged['VENTA_INCREMENTAL_TOTAL'] = merged['VENTA_INCREMENTAL'] + (venta_incremental_por_asignar * merged['PORCENTAJE_VENTA_INCREMENTAL'])

print(merged[['CLV_PROMOCION', 'CLV_CONTROL', 'LIFT', 'VENTA_INCREMENTAL_TOTAL']])

         CLV_PROMOCION  CLV_CONTROL      LIFT VENTA_INCREMENTAL_TOTAL
SEGMENTO                                                             
HIGH        268.125853   253.089846   0.05941          2263159.314886
LOW         167.783659   165.822142  0.011829             112826.7164
MEDIUM      190.745946   187.662337  0.016432           363705.635758
TOP         433.896021   413.518398  0.049279           609406.152133


## Medimos los otros datos por segmento

In [16]:
cur_otros_datos_por_segmento = conn.cursor().execute(f"""
    WITH

    INTERVALO_FECHAS AS (
        -- Fecha de inicio y final que vamos a usar para la segmentacion
        SELECT
            to_date({FECHA_INICIO_ACTIVOS}) AS FECHA_INICIO,
            to_date({FECHA_FIN}) AS FECHA_FIN
    ),

    TRANSACCIONES_BY_USER AS (
        SELECT
            EMAIL,
            sum(CHECK_AMOUNT / 1.16) AS VENTAS,
            count(DISTINCT TRANSACTION_ID) AS FREQ,
            VENTAS / FREQ AS AOV
        FROM
            SEGMENT_EVENTS.SESSIONM_SBX.FACT_TRANSACTIONS
        INNER JOIN
            INTERVALO_FECHAS
        ON
            to_date(CREATED_AT) BETWEEN FECHA_INICIO AND FECHA_FIN
        WHERE
            EMAIL IS NOT null
        GROUP BY
            EMAIL
    ),

    SEGMENTACION_BASE_ACTIVOS AS (
        SELECT 
            EMAIL,
            CASE
                WHEN FREQ <= PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'LIGHT'
                WHEN FREQ <= PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'MIDL'
                WHEN FREQ <= PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'MIDH'
                WHEN FREQ <= PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'HEAVY'
                ELSE 'SUPER'
            END AS SEGMENTO,
            CASE
                WHEN AOV <= ROUND(PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY AOV) OVER ()) THEN 'LOW'
                WHEN AOV <= ROUND(PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY AOV) OVER ()) THEN 'AVG'
                ELSE 'HIGH'
            END AS TAG_GRUPO,
            SEGMENTO || '_' || TAG_GRUPO AS FULL_SEGMENTO,
            CASE
                WHEN FULL_SEGMENTO IN ('LIGHT_LOW', 'LIGHT_AVG', 'LIGHT_HIGH', 'MIDL_LOW', 'MIDL_AVG', 'MIDH_LOW') THEN 'LOW'
                WHEN FULL_SEGMENTO IN ('MIDL_HIGH', 'MIDH_AVG', 'MIDH_HIGH', 'HEAVY_LOW', 'HEAVY_AVG') THEN 'MEDIUM' 
                WHEN FULL_SEGMENTO IN ('HEAVY_HIGH', 'SUPER_LOW', 'SUPER_AVG') THEN 'HIGH'
                ELSE 'TOP'
            END AS ULTRASEGMENTO
        FROM
            TRANSACCIONES_BY_USER
    ),

    TRANSACCIONES_TOTALES AS (
        SELECT 
            SEM_ALSEA, 
            ANIO_ALSEA,
            CHECK_AMOUNT/1.16 AS VENTA, 
            TRANSACTION_ID,
            EMAIL
        FROM 
            "SEGMENT_EVENTS"."SESSIONM_SBX"."FACT_TRANSACTIONS" A
        INNER JOIN 
            "WOW_REWARDS"."WORK_SPACE_WOW_REWARDS"."DS_DIM_TIME" T
        ON 
            to_date(CREATED_AT) = T.FECHA
        WHERE
            ANIO_ALSEA = {ANIO_MEDICION}
        AND
            SEM_ALSEA = {SEMANA_MEDICION}    
    )

    SELECT
        sum(VENTA) AS VENTAS,
        count(DISTINCT TRANSACTION_ID) AS ORDENES,
        count(DISTINCT TRANSACCIONES_TOTALES.EMAIL) AS USUARIOS,
        count(DISTINCT SEGMENTACION.EMAIL) AS USUARIOS_TOTALES_SEGMENTACION,
        ULTRASEGMENTO,
        VENTAS/USUARIOS AS CLV,
        VENTAS/ORDENES AS TICKET_PROMEDIO,
        ORDENES/USUARIOS AS FRECUENCIA
    FROM
        SEGMENTACION_BASE_ACTIVOS AS SEGMENTACION
    LEFT JOIN
        TRANSACCIONES_TOTALES
    ON
        SEGMENTACION.EMAIL = TRANSACCIONES_TOTALES.EMAIL
    GROUP BY
        ULTRASEGMENTO
    ORDER BY
        CASE ULTRASEGMENTO
            WHEN 'LOW' THEN 1
            WHEN 'MEDIUM' THEN 2
            WHEN 'HIGH' THEN 3
            WHEN 'TOP' THEN 4
        END
    ;
""")

otros_datos_por_segmento = cur_otros_datos_por_segmento.fetch_pandas_all()
print(otros_datos_por_segmento)

              VENTAS  ORDENES  USUARIOS  USUARIOS_TOTALES_SEGMENTACION  \
0   9544309.99138032    81986     68832                         440357   
1  25762550.26724365   202566    139655                         347867   
2  43340818.63794840   436856    156364                         205983   
3  13060357.62069469    65991     26338                          33034   

  ULTRASEGMENTO               CLV   TICKET_PROMEDIO FRECUENCIA  
0           LOW  138.660942459616  116.413899828999   1.191103  
1        MEDIUM  184.472809904720  127.181018864191   1.450474  
2          HIGH  277.179009477555   99.210766563692   2.793840  
3           TOP  495.875071026452  197.911194264289   2.505543  


## Medimos los otros datos sin segmento

In [17]:
cur_otros_datos_sin_segmento = conn.cursor().execute(f"""
    WITH

    INTERVALO_FECHAS AS (
        -- Fecha de inicio y final que vamos a usar para la segmentacion
        SELECT
            to_date({FECHA_INICIO_ACTIVOS}) AS FECHA_INICIO,
            to_date({FECHA_FIN}) AS FECHA_FIN
    ),

    TRANSACCIONES_BY_USER AS (
        SELECT
            EMAIL,
            sum(CHECK_AMOUNT / 1.16) AS VENTAS,
            count(DISTINCT TRANSACTION_ID) AS FREQ,
            VENTAS / FREQ AS AOV
        FROM
            SEGMENT_EVENTS.SESSIONM_SBX.FACT_TRANSACTIONS
        INNER JOIN
            INTERVALO_FECHAS
        ON
            to_date(CREATED_AT) BETWEEN FECHA_INICIO AND FECHA_FIN
        WHERE
            EMAIL IS NOT null
        GROUP BY
            EMAIL
    ),

    SEGMENTACION_BASE_ACTIVOS AS (
        SELECT 
            EMAIL,
            CASE
                WHEN FREQ <= PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'LIGHT'
                WHEN FREQ <= PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'MIDL'
                WHEN FREQ <= PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'MIDH'
                WHEN FREQ <= PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY FREQ) OVER () THEN 'HEAVY'
                ELSE 'SUPER'
            END AS SEGMENTO,
            CASE
                WHEN AOV <= ROUND(PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY AOV) OVER ()) THEN 'LOW'
                WHEN AOV <= ROUND(PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY AOV) OVER ()) THEN 'AVG'
                ELSE 'HIGH'
            END AS TAG_GRUPO,
            SEGMENTO || '_' || TAG_GRUPO AS FULL_SEGMENTO,
            CASE
                WHEN FULL_SEGMENTO IN ('LIGHT_LOW', 'LIGHT_AVG', 'LIGHT_HIGH', 'MIDL_LOW', 'MIDL_AVG', 'MIDH_LOW') THEN 'LOW'
                WHEN FULL_SEGMENTO IN ('MIDL_HIGH', 'MIDH_AVG', 'MIDH_HIGH', 'HEAVY_LOW', 'HEAVY_AVG') THEN 'MEDIUM' 
                WHEN FULL_SEGMENTO IN ('HEAVY_HIGH', 'SUPER_LOW', 'SUPER_AVG') THEN 'HIGH'
                ELSE 'TOP'
            END AS ULTRASEGMENTO
        FROM
            TRANSACCIONES_BY_USER
    ),

    TRANSACCIONES_TOTALES AS (
        SELECT 
            SEM_ALSEA, 
            ANIO_ALSEA,
            CHECK_AMOUNT/1.16 AS VENTA, 
            TRANSACTION_ID,
            EMAIL
        FROM 
            "SEGMENT_EVENTS"."SESSIONM_SBX"."FACT_TRANSACTIONS" A
        INNER JOIN 
            "WOW_REWARDS"."WORK_SPACE_WOW_REWARDS"."DS_DIM_TIME" T
        ON 
            to_date(CREATED_AT) = T.FECHA
        WHERE
            ANIO_ALSEA = {ANIO_MEDICION}
        AND
            SEM_ALSEA = {SEMANA_MEDICION}    
    )

    SELECT
        sum(VENTA) AS VENTAS,
        count(DISTINCT TRANSACTION_ID) AS ORDENES,
        count(DISTINCT TRANSACCIONES_TOTALES.EMAIL) AS USUARIOS,
        count(DISTINCT SEGMENTACION.EMAIL) AS USUARIOS_TOTALES_SEGMENTACION,
        'TOTAL' AS ULTRASEGMENTO,
        VENTAS/USUARIOS AS CLV,
        VENTAS/ORDENES AS TICKET_PROMEDIO,
        ORDENES/USUARIOS AS FRECUENCIA
    FROM
        SEGMENTACION_BASE_ACTIVOS AS SEGMENTACION
    LEFT JOIN
        TRANSACCIONES_TOTALES
    ON
        SEGMENTACION.EMAIL = TRANSACCIONES_TOTALES.EMAIL
    ;
""")

otros_datos_sin_segmento = cur_otros_datos_sin_segmento.fetch_pandas_all()
print(otros_datos_sin_segmento)

              VENTAS  ORDENES  USUARIOS  USUARIOS_TOTALES_SEGMENTACION  \
0  91708036.51726706   787399    391189                        1027241   

  ULTRASEGMENTO               CLV   TICKET_PROMEDIO FRECUENCIA  
0         TOTAL  234.434088170340  116.469587232479   2.012835  


## Unimos los datos con y sin segmento, y los ponemos en un archivo csv

In [18]:
otros_datos_total = concat([otros_datos_sin_segmento, otros_datos_por_segmento])[['ULTRASEGMENTO', 'USUARIOS_TOTALES_SEGMENTACION', 'TICKET_PROMEDIO', 'FRECUENCIA', 'CLV']]
print(otros_datos_total)
otros_datos_total.to_csv('results' + FECHA_FIN + '.csv', index = False)

  ULTRASEGMENTO  USUARIOS_TOTALES_SEGMENTACION   TICKET_PROMEDIO FRECUENCIA  \
0         TOTAL                        1027241  116.469587232479   2.012835   
0           LOW                         440357  116.413899828999   1.191103   
1        MEDIUM                         347867  127.181018864191   1.450474   
2          HIGH                         205983   99.210766563692   2.793840   
3           TOP                          33034  197.911194264289   2.505543   

                CLV  
0  234.434088170340  
0  138.660942459616  
1  184.472809904720  
2  277.179009477555  
3  495.875071026452  
